In [1]:
import pandas as pd
import pareto
import os
import sys
from itertools import chain
from matplotlib import pyplot as plt
sys.path.append('..')

In [2]:
import rbf_functions

# Load solutions for each RBF for all seeds:

In [3]:
import os

rbfs = [rbf_functions.squared_exponential_rbf,
        rbf_functions.original_rbf,
        rbf_functions.inverse_quadratic_rbf,
        rbf_functions.inverse_multiquadric_rbf,
        rbf_functions.exponential_rbf,
        rbf_functions.matern32_rbf,
        rbf_functions.matern52_rbf,
       ]

root_dir = "../output_smaller_eps/"
pareto_sets = {}
for entry in rbfs:
    sollist = []
    solutions = []
    name = entry.__name__
    output_dir = os.path.join(root_dir, name)
    
    for filename in os.listdir(output_dir):
        if filename.endswith('solution.csv'):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}/{filename}", header=0)
            solutions.append(df_temp.values.tolist())
    pareto_sets[name] = list(chain.from_iterable(solutions))

pareto_sets.keys()
print("Amount of solutions for each RBF:")
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))
# print(f"Loaded into list 'solutions': {', '.join(sollist)}")

Amount of solutions for each RBF:
squared_exponential_rbf 323
original_rbf 375
inverse_quadratic_rbf 321
inverse_multiquadric_rbf 364
exponential_rbf 370
matern32_rbf 327
matern52_rbf 371


# Create reference set for each RBF, and save:

In [4]:
reference_sets = {}
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))
    nondominated = pareto.eps_sort([pareto_sets[rbf]], [0,1,2,3], [0.05, 0.05, 0.05, 0.05], maximize=[1,2,3])
    reference_sets[rbf] = nondominated
    df_nondom = pd.DataFrame(nondominated, columns=['max_p', 'utility', 'inertia','reliability'])
    print(rbf, len(df_nondom))
    df_nondom.to_csv(f"./{rbf}_refset.csv", index=False, header=True)


squared_exponential_rbf 323
squared_exponential_rbf 30
original_rbf 375
original_rbf 32
inverse_quadratic_rbf 321
inverse_quadratic_rbf 31
inverse_multiquadric_rbf 364
inverse_multiquadric_rbf 30
exponential_rbf 370
exponential_rbf 31
matern32_rbf 327
matern32_rbf 30
matern52_rbf 371
matern52_rbf 32


# Find decision variables that belong to the generated refset:

In [5]:
d_refvar = {}
d_refsol = {}
for entry in rbfs:
    name = entry.__name__
    output_dir = os.path.join(root_dir, name)
    
    ts = []
    tv = []
    for filename in os.listdir(output_dir):
        #load 
        if filename.endswith('solution.csv'):
            df_ts = pd.read_csv(f"{output_dir}/{filename}", header=0)
            ts.append(df_ts)
        elif filename.endswith('variables.csv'):
            df_tv = pd.read_csv(f"{output_dir}/{filename}", header=None)
            tv.append(df_tv)
    
    df_sol = pd.concat(ts)
    df_var = pd.concat(tv)
    
    sol = df_sol.values.tolist()
    var = df_var.values.tolist()
    ref = reference_sets[name]
    refvar = []
    refsol = []
    for idx, value in enumerate(sol):
        if value in ref:
            refvar.append(var[idx])
            refsol.append(sol[idx])
    d_refsol[name] = pd.DataFrame(refsol, columns=['max_p', 'utility', 'inertia','reliability'])
    d_refvar[name] = pd.DataFrame(refvar)
    d_refsol[name].to_csv(f"./refsets/{name}_refset.csv", index=False, header=True)
    d_refvar[name].to_csv(f"./refsets/{name}_refset_variables.csv", index=False, header=False)

In [6]:
for entry in rbfs:
    name = entry.__name__
    print(name)
    print(f"refset: {len(d_refsol[name])}")
    print(f"varset: {len(d_refvar[name])}")

squared_exponential_rbf
refset: 30
varset: 30
original_rbf
refset: 32
varset: 32
inverse_quadratic_rbf
refset: 31
varset: 31
inverse_multiquadric_rbf
refset: 30
varset: 30
exponential_rbf
refset: 31
varset: 31
matern32_rbf
refset: 30
varset: 30
matern52_rbf
refset: 32
varset: 32


# Generate global reference set for all RBFs:

In [7]:
x = 0
for rbf in pareto_sets:
    x += len(pareto_sets[rbf])
    print(rbf, len(pareto_sets[rbf]))
print("total:", x)
    
pareto_set = {}
sollist = []
solutions = []
for entry in rbfs:
    name = entry.__name__
    output_dir = os.path.join(root_dir, name)
    for filename in os.listdir(output_dir):
        if filename.endswith('solution.csv'):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}/{filename}", header=0)
            solutions.append(df_temp.values.tolist())
pareto_set = list(chain.from_iterable(solutions))
len(pareto_set)

squared_exponential_rbf 323
original_rbf 375
inverse_quadratic_rbf 321
inverse_multiquadric_rbf 364
exponential_rbf 370
matern32_rbf 327
matern52_rbf 371
total: 2451


2451

In [10]:
print(len(pareto_set))
nondominated = pareto.eps_sort([pareto_set], [0,1,2,3], [0.05, 0.05, 0.05, 0.05], maximize=[1,2,3])
df_nondom = pd.DataFrame(nondominated, columns=['max_p', 'utility', 'inertia','reliability'])
print(len(nondominated))
df_nondom.to_csv(f"./refsets/all_rbf_refset.csv", index=False, header=True)

2451
29


In [11]:
pareto_set

[[0.1928013489833658,
  0.5197030545705805,
  0.9900000000000014,
  1.0000000000000009],
 [0.2284350234448139,
  0.5528580110810631,
  0.9900000000000014,
  1.0000000000000009],
 [1.2536272014515517,
  1.123099609319649,
  0.9834000000000012,
  0.5395000000000006],
 [1.397274563520015, 1.20506518148524, 0.9449000000000004, 0.4743000000000003],
 [0.3275892060827505,
  0.6054096264832378,
  0.9880000000000012,
  0.9550000000000006],
 [0.9637735306549288,
  0.9571886225963512,
  0.979900000000001,
  0.6702000000000008],
 [0.511728195958862,
  0.7059328006343288,
  0.9880000000000012,
  0.8726000000000007],
 [0.8605302539584713,
  0.9009453938039096,
  0.9768000000000012,
  0.7156000000000008],
 [1.6653259688178488,
  1.3580376291195024,
  0.9781000000000012,
  0.3528000000000002],
 [1.2319130921391264,
  1.1108420569155997,
  0.9237000000000004,
  0.5481000000000004],
 [2.2427969215215837, 1.7085983689284856, 0.9835000000000012, 0.0887],
 [0.4172623933602691,
  0.6540128249241213,
  0.987